# Driver Overview

Let's run the WPS prep utility `ungrib` to transform a GFS 0-hour forecast GRIB files into initial conditions suitable for WRF or MPAS.

First `cd` into the `configs/` directory:

In [ ]:
import os, shutil
os.chdir("../configs")
shutil.rmtree("driver-overview", ignore_errors=True)

Here's the config we'll use. Note the additional `platform:` and `ungrib.execution.batchargs:` blocks, which will be necessary for running on the batch system:

In [ ]:
!cat do00.yaml

First, use `uw fs` to stage the GFS GRIB file from AWS:

In [ ]:
!uw fs copy -c do00.yaml --key-path data --cycle 2025-09-21T06

Now run `ungrib` via the UW driver, locally:

In [ ]:
!uw ungrib run -c do00.yaml --cycle 2025-09-21T06

See the output:

In [ ]:
!tree driver-overview

Note that an invocation again is a no-op:

In [ ]:
!uw ungrib run -c do00.yaml --cycle 2025-09-21T06

The `ungrib` driver views its `runscript.ungrib.done` file as evidence of its readiness, so we'll need to remove it to force re-execution. We'll also remove `runscript.ungrib` so that it gets recreated with batch directives -- remember, drivers are idempotent, and `runscript.ungrib` already exists. For brevity, we'll just remove `runscript.ungrib*`.

In [ ]:
%%bash
rm -fv driver-overview/runscript.ungrib*
uw ungrib run -c do00.yaml --cycle 2025-09-21T06 --batch
squeue -u $USER

After a short time...

In [ ]:
%%bash
squeue -u $USER
tree driver-overview

Note the new batch directives in `runscript.ungrib`:

In [ ]:
!cat driver-overview/runscript.ungrib

We can also execute just parts of the driver's task graph. Let's see the available tasks:

In [ ]:
!uw ungrib --help

Let's remove the run directory and this time ask just for the namelist file:

In [ ]:
%%bash
rm -rf driver-overview
uw ungrib namelist_file -c do00.yaml --cycle 2025-09-21T06

In [ ]:
!tree driver-overview

In [ ]:
cat driver-overview/namelist.wps

Expand that request to the full provisioned run directory:

In [ ]:
!uw ungrib provisioned_rundir -c do00.yaml --cycle 2025-09-21T06

Note two interesting things:
1. The namelist file was not recreated, because it was created by the previous invocation.
2. Since we removed the full run directory, including the GFS GRIB file, `uw` reports that the `provisioned_rundir` task is not complete: It needs the GRIB file to be complete. We can stage that, then re-run the driver:

In [ ]:
!uw fs copy -c do00.yaml --key-path data --cycle 2025-09-21T06

In [ ]:
!uw ungrib provisioned_rundir -c do00.yaml --cycle 2025-09-21T06

Now the run directory is fully provisioned. We could run ungrib with `uw ungrib run`, or even run its `runscript.ungrib` directly. For experimental purposes, we could even manually edit any of the run-directory files first, then execute the run.

In [ ]:
!rm -rf driver-overview